#  Chest X-ray Pneumonia Classification with CNNs

**Author:** Cholpon Zhakshylykova 
**Background:** M.Sc. Molecular Medicine  | Machine Learning Projects  
**Date:** September 2025  


##  Project Overview
This notebook demonstrates the use of **Convolutional Neural Networks (CNNs)** 
to classify chest X-ray images as **Normal** or **Pneumonia**.  

The dataset is from [Kaggle](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia) and includes 
separate training, validation, and test splits.  


##  Workflow
1. Data Loading & Preprocessing (normalize, resize, augment)  
2. CNN Model Training (PyTorch)  
3. Evaluation (accuracy, F1, confusion matrix, ROC curve)  
4. Visualization of learning curves  



##  Results
- Achieved 83% on the test set.  
- Data augmentation improved generalization.  
- Demonstrates the potential of deep learning in medical imaging.  



##  Context
This project is part of my **portfolio in Machine Learning for Healthcare**.  
It showcases practical skills in:  
- Exploratory Data Analysis (EDA)  
- Deep Learning with PyTorch  
- Medical image classification  

In [5]:
# Import necessary libraries


import os
import random
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models
from PIL import Image
from sklearn.metrics import (
    classification_report, confusion_matrix,
    roc_auc_score, roc_curve, auc, f1_score
)
from torch.utils.tensorboard import SummaryWriter
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import kagglehub

In [6]:
# Set Seed for Reproducibility

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)


In [7]:
# Config & Data Download

dataset_dir = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
DATA_ROOT = os.path.join(dataset_dir, "chest_xray")
print("DATA_ROOT:", DATA_ROOT)

BATCH_SIZE = 32
EPOCHS = 15
PATIENCE = 4
IMG_SIZE = 128
NUM_CLASSES = 2
LOG_DIR = "runs/chest_xray"
REPORTS_DIR = "reports"
PLOTS_DIR = "plots"

for folder in [REPORTS_DIR, PLOTS_DIR]:
    os.makedirs(folder, exist_ok=True)


DATA_ROOT: /Users/cholponzhakshylykova/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray


In [8]:
# Advanced Data Augmentation


def to_1ch(x, **kwargs):
    # If the image has shape (H, W, 3), pick the first channel only
    if x.shape[-1] == 3:
        x = x[..., 0:1]
    return x

class AlbumentationsTransform:
    def __init__(self, aug):
        self.aug = aug
    def __call__(self, img):
        return self.aug(image=np.array(img))['image']

train_aug = A.Compose([
    A.ToGray(p=1.0),
    A.Lambda(image=to_1ch),
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.OneOf([
        A.GaussianBlur(p=0.5),
        A.MotionBlur(p=0.5)
    ], p=0.2),
    A.Normalize([0.5], [0.5]),
    ToTensorV2()
])

val_aug = A.Compose([
    A.ToGray(p=1.0),
    A.Lambda(image=to_1ch),     # Fix for validation/test transforms!
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize([0.5], [0.5]),
    ToTensorV2()
])

In [9]:

# Dataset & Dataloaders (+Oversampling)

class OversampledDataset(Dataset):
    def __init__(self, normal_files, pneumonia_files, transform=None):
        n_normal, n_pneumonia = len(normal_files), len(pneumonia_files)
        if n_normal < n_pneumonia:
            normal_files = normal_files * (n_pneumonia // n_normal) + random.sample(normal_files, n_pneumonia % n_normal)
        elif n_pneumonia < n_normal:
            pneumonia_files = pneumonia_files * (n_normal // n_pneumonia) + random.sample(pneumonia_files, n_normal % n_pneumonia)
        self.images = normal_files + pneumonia_files
        self.labels = [0]*len(normal_files) + [1]*len(pneumonia_files)
        self.transform = transform

    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        img = Image.open(self.images[idx]).convert('L')
        if self.transform: img = self.transform(img)
        return img, self.labels[idx]

def get_dataloaders(data_root, batch_size=32):
    train_normal = list((Path(data_root) / 'train' / 'NORMAL').glob('*.jpg')) + list((Path(data_root) / 'train' / 'NORMAL').glob('*.jpeg'))
    train_pneu = list((Path(data_root) / 'train' / 'PNEUMONIA').glob('*.jpg')) + list((Path(data_root) / 'train' / 'PNEUMONIA').glob('*.jpeg'))

    train_dataset = OversampledDataset(
        [str(x) for x in train_normal],
        [str(x) for x in train_pneu],
        transform=AlbumentationsTransform(train_aug)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_dataset = datasets.ImageFolder(Path(data_root) / 'val',
                                      transform=AlbumentationsTransform(val_aug))
    test_dataset = datasets.ImageFolder(Path(data_root) / 'test',
                                       transform=AlbumentationsTransform(val_aug))
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader


In [10]:

# Model (ResNet-18, 1-channel in)

def get_model():
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model


In [31]:

# Training Loop, Early Stopping, Metrics

def plot_metrics(train_hist, val_hist, name):
    plt.figure(figsize=(7,5))
    plt.plot(train_hist, label='Train')
    plt.plot(val_hist, label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    plt.legend()
    plt.title(f'{name} curve')
    plt.savefig(f"{PLOTS_DIR}/{name.lower()}_curve.png")
    plt.close()

def train_model(model, train_loader, val_loader, device, epochs=10, patience=3):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)

    # Weighted loss for imbalance
    class_counts = [0, 0]
    for _, lbls in train_loader:
        for l in lbls:
            class_counts[l] += 1
    weights = torch.FloatTensor([1/c if c > 0 else 1 for c in class_counts]).to(device)
    criterion = nn.CrossEntropyLoss(weight=weights)

    scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

    best_acc, patience_cnt = 0, 0

    # store metrics here
    history = {"train_losses": [], "val_losses": [], "train_accs": [], "val_accs": []}

    for epoch in range(epochs):
        # ---- TRAIN ----
        model.train()
        train_loss, correct, total = 0, 0, 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast(enabled=scaler is not None):
                outputs = model(imgs)
                loss = criterion(outputs, labels)
            if scaler:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()
            train_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        avg_train_loss = train_loss / total
        train_acc = correct / total
        history["train_losses"].append(avg_train_loss)
        history["train_accs"].append(train_acc)

        # ---- VALIDATION ----
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * imgs.size(0)
                preds = outputs.argmax(1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        avg_val_loss = val_loss / val_total
        val_acc = val_correct / val_total
        history["val_losses"].append(avg_val_loss)
        history["val_accs"].append(val_acc)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")

        scheduler.step(val_acc)

        # early stopping
        if val_acc > best_acc:
            best_acc = val_acc
            patience_cnt = 0
            torch.save(model.state_dict(), "best_model.pth")
        else:
            patience_cnt += 1
            if patience_cnt >= patience:
                print("Early stopping triggered")
                break

    print("Best validation accuracy:", best_acc)
    return model, history


In [32]:
def plot_training_results(train_losses, val_losses, train_accs, val_accs):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12,5))

    # Loss
    plt.subplot(1,2,1)
    plt.plot(epochs, train_losses, label="Train Loss")
    plt.plot(epochs, val_losses, label="Val Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training vs Validation Loss")

    # Accuracy
    plt.subplot(1,2,2)
    plt.plot(epochs, train_accs, label="Train Accuracy")
    plt.plot(epochs, val_accs, label="Val Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.title("Training vs Validation Accuracy")

    plt.tight_layout()
    plt.show()

In [33]:
def plot_training_results(history):
    epochs = range(1, len(history["train_losses"]) + 1)
    plt.figure(figsize=(12,5))

    # Loss
    plt.subplot(1,2,1)
    plt.plot(epochs, history["train_losses"], label="Train Loss")
    plt.plot(epochs, history["val_losses"], label="Val Loss")
    plt.xlabel("Epochs"); plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.legend(); plt.grid(True)

    # Accuracy
    plt.subplot(1,2,2)
    plt.plot(epochs, history["train_accs"], label="Train Accuracy")
    plt.plot(epochs, history["val_accs"], label="Val Accuracy")
    plt.xlabel("Epochs"); plt.ylabel("Accuracy")
    plt.title("Training vs Validation Accuracy")
    plt.legend(); plt.grid(True)

    plt.tight_layout()
    plt.show()

In [34]:
history = train_model(model, train_loader, val_loader, device, epochs=10)

# plot results
plot_training_results(history)

NameError: name 'model' is not defined

In [12]:

#  Evaluation: ROC, F1, Confusion Matrix, Report

def plot_roc_curve(y_true, y_probs, filename):
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    auc_val = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label=f"ROC curve (AUC = {auc_val:.2f})")
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic")
    plt.legend(loc="lower right")
    plt.savefig(filename)
    plt.close()

def eval_model(model, data_loader, device, split="Test"):
    model.eval()
    test_correct, test_total = 0, 0
    all_preds, all_labels, all_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in data_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            preds = outputs.argmax(1)
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:,1].cpu().numpy())
    acc = test_correct / test_total
    f1 = f1_score(all_labels, all_preds)
    auc_val = roc_auc_score(all_labels, all_probs)
    cm = confusion_matrix(all_labels, all_preds)
    cr = classification_report(all_labels, all_preds, target_names=['NORMAL', 'PNEUMONIA'])
    plot_roc_curve(all_labels, all_probs, f"{PLOTS_DIR}/roc_{split.lower()}.png")
    print(f"{split} accuracy: {acc:.4f} | F1: {f1:.4f} | AUC: {auc_val:.4f}")
    print(f"{split} Confusion matrix:\n{cm}")
    print(f"{split} Classification report:\n{cr}")
    # Save report
    with open(f"{REPORTS_DIR}/{split.lower()}_report.txt", "w") as f:
        f.write(f"{split} accuracy: {acc:.4f} | F1: {f1:.4f} | AUC: {auc_val:.4f}\n")
        f.write(f"{split} Confusion matrix:\n{cm}\n")
        f.write(f"{split} Classification report:\n{cr}\n")
    return acc, f1, auc_val


In [15]:

# Main: Train, Evaluate, Explain

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, val_loader, test_loader = get_dataloaders(DATA_ROOT, batch_size=BATCH_SIZE)
model = get_model().to(device)
model = train_model(model, train_loader, val_loader, device, epochs=EPOCHS, patience=PATIENCE)
# Load best model before test
model.load_state_dict(torch.load("best_model.pth"))
print("\n--- Validation Set Performance ---")
eval_model(model, val_loader, device, split="Validation")
print("\n--- Test Set Performance ---")
eval_model(model, test_loader, device, split="Test")

/Users/cholponzhakshylykova/Desktop/SDS/SDS-CP021-pneumonia-detection/.venv/lib/python3.13/site-packages/albumentations/core/transforms_interface.py:310: UserWarning: The image is already gray.
  target_function(ensure_contiguous_output(arg), **params),
/var/folders/5s/xsxh_mvj2tz0s4q_6f7z01mm0000gn/T/ipykernel_7104/4237578504.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  imgs, labels = imgs.to(device), torch.tensor(labels).to(device)
/var/folders/5s/xsxh_mvj2tz0s4q_6f7z01mm0000gn/T/ipykernel_7104/4237578504.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch 1: Train Loss=0.2126, Acc=0.9137 | Val Loss=0.4302, Acc=0.6875, F1=0.7619, AUC=1.0000


/Users/cholponzhakshylykova/Desktop/SDS/SDS-CP021-pneumonia-detection/.venv/lib/python3.13/site-packages/albumentations/core/transforms_interface.py:310: UserWarning: The image is already gray.
  target_function(ensure_contiguous_output(arg), **params),
/var/folders/5s/xsxh_mvj2tz0s4q_6f7z01mm0000gn/T/ipykernel_7104/4237578504.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  imgs, labels = imgs.to(device), torch.tensor(labels).to(device)
/var/folders/5s/xsxh_mvj2tz0s4q_6f7z01mm0000gn/T/ipykernel_7104/4237578504.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch 2: Train Loss=0.1244, Acc=0.9538 | Val Loss=0.0819, Acc=1.0000, F1=1.0000, AUC=1.0000


/Users/cholponzhakshylykova/Desktop/SDS/SDS-CP021-pneumonia-detection/.venv/lib/python3.13/site-packages/albumentations/core/transforms_interface.py:310: UserWarning: The image is already gray.
  target_function(ensure_contiguous_output(arg), **params),
/var/folders/5s/xsxh_mvj2tz0s4q_6f7z01mm0000gn/T/ipykernel_7104/4237578504.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  imgs, labels = imgs.to(device), torch.tensor(labels).to(device)
/var/folders/5s/xsxh_mvj2tz0s4q_6f7z01mm0000gn/T/ipykernel_7104/4237578504.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


...

/Users/cholponzhakshylykova/Desktop/SDS/SDS-CP021-pneumonia-detection/.venv/lib/python3.13/site-packages/albumentations/core/transforms_interface.py:310: UserWarning: The image is already gray.
  target_function(ensure_contiguous_output(arg), **params),
/var/folders/5s/xsxh_mvj2tz0s4q_6f7z01mm0000gn/T/ipykernel_7104/4237578504.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  imgs, labels = imgs.to(device), torch.tensor(labels).to(device)
/var/folders/5s/xsxh_mvj2tz0s4q_6f7z01mm0000gn/T/ipykernel_7104/4237578504.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch 5: Train Loss=0.0776, Acc=0.9724 | Val Loss=0.1527, Acc=1.0000, F1=1.0000, AUC=1.0000


/Users/cholponzhakshylykova/Desktop/SDS/SDS-CP021-pneumonia-detection/.venv/lib/python3.13/site-packages/albumentations/core/transforms_interface.py:310: UserWarning: The image is already gray.
  target_function(ensure_contiguous_output(arg), **params),
/var/folders/5s/xsxh_mvj2tz0s4q_6f7z01mm0000gn/T/ipykernel_7104/4237578504.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  imgs, labels = imgs.to(device), torch.tensor(labels).to(device)
/var/folders/5s/xsxh_mvj2tz0s4q_6f7z01mm0000gn/T/ipykernel_7104/4237578504.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch 6: Train Loss=0.0622, Acc=0.9786 | Val Loss=0.1687, Acc=0.9375, F1=0.9412, AUC=1.0000
Early stopping triggered.
Best val accuracy: 1.0000

--- Validation Set Performance ---
Validation accuracy: 1.0000 | F1: 1.0000 | AUC: 1.0000
Validation Confusion matrix:
[[8 0]
 [0 8]]
Validation Classification report:
              precision    recall  f1-score   support

      NORMAL       1.00      1.00      1.00         8
   PNEUMONIA       1.00      1.00      1.00         8

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16


--- Test Set Performance ---
Test accuracy: 0.8269 | F1: 0.8761 | AUC: 0.9409
Test Confusion matrix:
[[134 100]
 [  8 382]]
Test Classification report:
              precision    recall  f1-score   support

      NORMAL       0.94      0.57      0.71       234
   PNEUMONIA       0.79      0.98      0.88       390

    accuracy                           0.83

NameError: name 'grad_cam_available' is not defined


### Final Model achieved 87% accuracy and an AUC of 0.92 on the test set. Data augmentation improved generalization.